In [ ]:
from sklearn.linear_model import LinearRegression
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR


df=yf.download('NVDA',start='2023-01-01',end='2024-01-01')
data=df.copy()
data=data.reset_index()
data['Day']=(data['Date']-data['Date'].min()).dt.days
data['Day']
data['MA_5']=data['Adj Close'].rolling(window=5).mean()
data['Volatility']=data['Adj Close'].rolling(window=5).std()

data.fillna(0,inplace=True)

x=data[['Day','MA_5','Volatility']]
y=data['Adj Close'].to_numpy().reshape(-1,1)

scx=StandardScaler()
scy=StandardScaler()
x=scx.fit_transform(x)

param_grid= {
    'C':[0.1,1,10,100],
    'gamma':[1,0.1,0.01,0.001],
    'epsilon':[0.1,0.01,0.001]
}
grid=GridSearchCV(SVR(kernel='rbf'),param_grid,cv=5)
grid.fit(x,y.ravel())
svrmodel2=grid.best_estimator_

y_pred=svrmodel2.predict(x)
rmse=np.sqrt(mean_squared_error(y,y_pred))
mae=mean_absolute_error(y,y_pred)
r2=r2_score(y,y_pred)
print(f'RMSE: {rmse},MAE:{mae}')
plt.figure(figsize=(10, 6))  
plt.scatter(data['Day'], y, color='red', label='Real Values') 
plt.plot(data['Day'], y_pred, color='blue', label='SVR Predictions')  
plt.title('Nvidia Stock Price Prediction(2023-2024)')
plt.xlabel('Day')
plt.ylabel('Adjusted Close Price')
plt.legend()
plt.show()


print(rmse,mae,r2)
